# BiLSTM을 이용한 영화 리뷰 감정 분석
Word2Vec을 통해 임베딩 된 네이버 영화 리뷰 데이터를 BiLSTM을 통해 긍정, 부정을 분류해주는 예제


## Train

In [1]:
import time
import os
import tensorflow as tf
import numpy as np
import Bi_LSTM
import Word2Vec

D:\Anaconda3\envs\dev-tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
DATA_PATH = "../data"
TRAIN_DATA = DATA_PATH + "/ratings_train.txt"
TEST_DATA = DATA_PATH + "/ratings_test.txt"
MODEL_DATA = DATA_PATH + "/Word2Vec.model"
META_DATA = DATA_PATH + "/metadata.tsv"
WORD2VEC_PATH = DATA_PATH + "/word2vec"

In [3]:
W2V = Word2Vec.Word2Vec()

In [4]:
train_data = W2V.read_data(TRAIN_DATA)

In [5]:
## tokenize the data we have
print("Tokenize Start!\nCould take minutes...")
tokens = [[W2V.tokenize(row[1]),int(row[2])] for row in train_data if W2V.tokenize(row[1]) != []]
tokens = np.array(tokens)
print("Tokenize Done!")

train_X = tokens[:,0]
train_Y = tokens[:,1]

Tokenize Start!
Could take minutes...


D:\Anaconda3\envs\dev-tensorflow\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


Tokenize Done!


In [6]:
train_Y_ = W2V.One_hot(train_Y)  ## Convert to One-hot
train_X_ = W2V.Convert2Vec(MODEL_DATA,train_X)  ## import word2vec model where you have trained before

- num_class : 긍정 or 부정 판별
- keep_prob : drop out rate

In [7]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300
seq_length = [len(x) for x in train_X]
Maxseq_length = max(seq_length) ## 95
learning_rate = 0.001
lstm_units = 128
num_class = 2
training_epochs = 10
keep_prob = 0.75

In [8]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

In [9]:
BiLSTM = Bi_LSTM.Bi_LSTM(lstm_units, num_class, keep_prob)

In [10]:
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

In [11]:
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
init = tf.global_variables_initializer()

In [13]:
total_batch = int(Total_size / Batch_size)

In [14]:
print("Start training!")
modelName = DATA_PATH + '/BiLSTM_model.ckpt'
saver = tf.train.Saver()

Start training!


In [15]:
with tf.Session() as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter(DATA_PATH, sess.graph)
    merged = BiLSTM.graph_build()
    
    for epoch in range(training_epochs):

        avg_acc, avg_loss = 0. , 0.
        for step in range(total_batch):

            train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
            train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
            batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
            
            train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
            
            sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            # Compute average loss
            loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_loss += loss_ / total_batch
            
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_acc += acc / total_batch
            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))
   
        summary = sess.run(merged, feed_dict = {BiLSTM.loss : avg_loss, BiLSTM.acc : avg_acc})       
        train_writer.add_summary(summary, epoch)
        
    train_writer.close()
    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, modelName)
    
    print ('save_path',save_path)
    
    ## cmd 실행 -> cd C:\Users\jbk48\Desktop\Sentimental-Analysis-master\Sentimental-Analysis-master\Bidirectional_LSTM
    ## tensorboard --logdir=./ 입력

epoch : 01 step : 0001 loss = 0.669777 accuracy= 0.562500
epoch : 01 step : 0002 loss = 0.675684 accuracy= 0.531250
epoch : 01 step : 0003 loss = 0.653625 accuracy= 0.687500
epoch : 01 step : 0004 loss = 0.675815 accuracy= 0.562500
epoch : 01 step : 0005 loss = 0.651864 accuracy= 0.593750
epoch : 01 step : 0006 loss = 0.705880 accuracy= 0.500000
epoch : 01 step : 0007 loss = 0.684243 accuracy= 0.500000
epoch : 01 step : 0008 loss = 0.645752 accuracy= 0.656250
epoch : 01 step : 0009 loss = 0.661986 accuracy= 0.531250
epoch : 01 step : 0010 loss = 0.644974 accuracy= 0.625000
epoch : 01 step : 0011 loss = 0.627666 accuracy= 0.687500
epoch : 01 step : 0012 loss = 0.617721 accuracy= 0.718750
epoch : 01 step : 0013 loss = 0.610513 accuracy= 0.750000
epoch : 01 step : 0014 loss = 0.525393 accuracy= 0.812500
epoch : 01 step : 0015 loss = 0.570227 accuracy= 0.656250
epoch : 01 step : 0016 loss = 0.661391 accuracy= 0.562500
epoch : 01 step : 0017 loss = 0.650099 accuracy= 0.625000
epoch : 01 ste

epoch : 01 step : 0143 loss = 0.414161 accuracy= 0.812500
epoch : 01 step : 0144 loss = 0.427131 accuracy= 0.781250
epoch : 01 step : 0145 loss = 0.494632 accuracy= 0.875000
epoch : 01 step : 0146 loss = 0.380186 accuracy= 0.812500
epoch : 01 step : 0147 loss = 0.282347 accuracy= 0.937500
epoch : 01 step : 0148 loss = 0.387878 accuracy= 0.843750
epoch : 01 step : 0149 loss = 0.422445 accuracy= 0.750000
epoch : 01 step : 0150 loss = 0.479677 accuracy= 0.718750
epoch : 01 step : 0151 loss = 0.431850 accuracy= 0.875000
epoch : 01 step : 0152 loss = 0.405207 accuracy= 0.875000
epoch : 01 step : 0153 loss = 0.496450 accuracy= 0.812500
epoch : 01 step : 0154 loss = 0.553421 accuracy= 0.781250
epoch : 01 step : 0155 loss = 0.280507 accuracy= 0.937500
epoch : 01 step : 0156 loss = 0.322574 accuracy= 0.875000
epoch : 01 step : 0157 loss = 0.329460 accuracy= 0.843750
epoch : 01 step : 0158 loss = 0.420134 accuracy= 0.781250
epoch : 01 step : 0159 loss = 0.553432 accuracy= 0.781250
epoch : 01 ste

epoch : 01 step : 0285 loss = 0.288767 accuracy= 0.875000
epoch : 01 step : 0286 loss = 0.466141 accuracy= 0.812500
epoch : 01 step : 0287 loss = 0.409263 accuracy= 0.843750
epoch : 01 step : 0288 loss = 0.520612 accuracy= 0.750000
epoch : 01 step : 0289 loss = 0.317984 accuracy= 0.875000
epoch : 01 step : 0290 loss = 0.411428 accuracy= 0.812500
epoch : 01 step : 0291 loss = 0.308443 accuracy= 0.937500
epoch : 01 step : 0292 loss = 0.385355 accuracy= 0.812500
epoch : 01 step : 0293 loss = 0.353489 accuracy= 0.812500
epoch : 01 step : 0294 loss = 0.280345 accuracy= 0.875000
epoch : 01 step : 0295 loss = 0.393885 accuracy= 0.781250
epoch : 01 step : 0296 loss = 0.342078 accuracy= 0.906250
epoch : 01 step : 0297 loss = 0.421847 accuracy= 0.750000
epoch : 01 step : 0298 loss = 0.364769 accuracy= 0.843750
epoch : 01 step : 0299 loss = 0.423117 accuracy= 0.843750
epoch : 01 step : 0300 loss = 0.377963 accuracy= 0.781250
epoch : 01 step : 0301 loss = 0.418074 accuracy= 0.781250
epoch : 01 ste

epoch : 01 step : 0427 loss = 0.477241 accuracy= 0.843750
epoch : 01 step : 0428 loss = 0.316137 accuracy= 0.937500
epoch : 01 step : 0429 loss = 0.287497 accuracy= 0.906250
epoch : 01 step : 0430 loss = 0.379587 accuracy= 0.781250
epoch : 01 step : 0431 loss = 0.586182 accuracy= 0.812500
epoch : 01 step : 0432 loss = 0.400853 accuracy= 0.781250
epoch : 01 step : 0433 loss = 0.495902 accuracy= 0.687500
epoch : 01 step : 0434 loss = 0.372099 accuracy= 0.875000
epoch : 01 step : 0435 loss = 0.375000 accuracy= 0.812500
epoch : 01 step : 0436 loss = 0.465642 accuracy= 0.812500
epoch : 01 step : 0437 loss = 0.357742 accuracy= 0.812500
epoch : 01 step : 0438 loss = 0.441926 accuracy= 0.781250
epoch : 01 step : 0439 loss = 0.345590 accuracy= 0.875000
epoch : 01 step : 0440 loss = 0.552212 accuracy= 0.718750
epoch : 01 step : 0441 loss = 0.388944 accuracy= 0.812500
epoch : 01 step : 0442 loss = 0.463345 accuracy= 0.812500
epoch : 01 step : 0443 loss = 0.307865 accuracy= 0.906250
epoch : 01 ste

epoch : 01 step : 0569 loss = 0.316921 accuracy= 0.875000
epoch : 01 step : 0570 loss = 0.321171 accuracy= 0.843750
epoch : 01 step : 0571 loss = 0.314421 accuracy= 0.843750
epoch : 01 step : 0572 loss = 0.267665 accuracy= 0.906250
epoch : 01 step : 0573 loss = 0.497765 accuracy= 0.718750
epoch : 01 step : 0574 loss = 0.275330 accuracy= 0.875000
epoch : 01 step : 0575 loss = 0.256991 accuracy= 0.875000
epoch : 01 step : 0576 loss = 0.360664 accuracy= 0.906250
epoch : 01 step : 0577 loss = 0.309520 accuracy= 0.843750
epoch : 01 step : 0578 loss = 0.417414 accuracy= 0.781250
epoch : 01 step : 0579 loss = 0.402636 accuracy= 0.843750
epoch : 01 step : 0580 loss = 0.479138 accuracy= 0.812500
epoch : 01 step : 0581 loss = 0.303626 accuracy= 0.906250
epoch : 01 step : 0582 loss = 0.411911 accuracy= 0.812500
epoch : 01 step : 0583 loss = 0.504198 accuracy= 0.781250
epoch : 01 step : 0584 loss = 0.499337 accuracy= 0.750000
epoch : 01 step : 0585 loss = 0.299720 accuracy= 0.875000
epoch : 01 ste

...

epoch : 10 step : 3988 loss = 0.022389 accuracy= 1.000000
epoch : 10 step : 3989 loss = 0.023332 accuracy= 1.000000
epoch : 10 step : 3990 loss = 0.081976 accuracy= 0.968750
epoch : 10 step : 3991 loss = 0.081424 accuracy= 0.937500
epoch : 10 step : 3992 loss = 0.001968 accuracy= 1.000000
epoch : 10 step : 3993 loss = 0.028322 accuracy= 1.000000
epoch : 10 step : 3994 loss = 0.100593 accuracy= 0.937500
epoch : 10 step : 3995 loss = 0.010926 accuracy= 1.000000
epoch : 10 step : 3996 loss = 0.051763 accuracy= 0.968750
epoch : 10 step : 3997 loss = 0.012629 accuracy= 1.000000
epoch : 10 step : 3998 loss = 0.034524 accuracy= 1.000000
epoch : 10 step : 3999 loss = 0.015834 accuracy= 1.000000
epoch : 10 step : 4000 loss = 0.026955 accuracy= 1.000000
epoch : 10 step : 4001 loss = 0.040105 accuracy= 1.000000
epoch : 10 step : 4002 loss = 0.011822 accuracy= 1.000000
epoch : 10 step : 4003 loss = 0.026970 accuracy= 1.000000
epoch : 10 step : 4004 loss = 0.039933 accuracy= 1.000000
epoch : 10 ste

epoch : 10 step : 4130 loss = 0.057217 accuracy= 1.000000
epoch : 10 step : 4131 loss = 0.093455 accuracy= 0.968750
epoch : 10 step : 4132 loss = 0.040919 accuracy= 1.000000
epoch : 10 step : 4133 loss = 0.030814 accuracy= 1.000000
epoch : 10 step : 4134 loss = 0.054229 accuracy= 0.968750
epoch : 10 step : 4135 loss = 0.039043 accuracy= 1.000000
epoch : 10 step : 4136 loss = 0.035262 accuracy= 1.000000
epoch : 10 step : 4137 loss = 0.029853 accuracy= 1.000000
epoch : 10 step : 4138 loss = 0.004366 accuracy= 1.000000
epoch : 10 step : 4139 loss = 0.007631 accuracy= 1.000000
epoch : 10 step : 4140 loss = 0.037144 accuracy= 0.968750
epoch : 10 step : 4141 loss = 0.026290 accuracy= 1.000000
epoch : 10 step : 4142 loss = 0.068998 accuracy= 0.968750
epoch : 10 step : 4143 loss = 0.018041 accuracy= 1.000000
epoch : 10 step : 4144 loss = 0.020399 accuracy= 1.000000
epoch : 10 step : 4145 loss = 0.011556 accuracy= 1.000000
epoch : 10 step : 4146 loss = 0.064518 accuracy= 0.968750
epoch : 10 ste

epoch : 10 step : 4273 loss = 0.016129 accuracy= 1.000000
epoch : 10 step : 4274 loss = 0.026550 accuracy= 1.000000
epoch : 10 step : 4275 loss = 0.093242 accuracy= 0.968750
epoch : 10 step : 4276 loss = 0.016777 accuracy= 1.000000
epoch : 10 step : 4277 loss = 0.116769 accuracy= 0.937500
epoch : 10 step : 4278 loss = 0.047800 accuracy= 1.000000
epoch : 10 step : 4279 loss = 0.066772 accuracy= 0.968750
epoch : 10 step : 4280 loss = 0.032921 accuracy= 1.000000
epoch : 10 step : 4281 loss = 0.007690 accuracy= 1.000000
epoch : 10 step : 4282 loss = 0.136577 accuracy= 0.937500
epoch : 10 step : 4283 loss = 0.018065 accuracy= 1.000000
epoch : 10 step : 4284 loss = 0.042037 accuracy= 1.000000
epoch : 10 step : 4285 loss = 0.013708 accuracy= 1.000000
epoch : 10 step : 4286 loss = 0.013438 accuracy= 1.000000
epoch : 10 step : 4287 loss = 0.003658 accuracy= 1.000000
epoch : 10 step : 4288 loss = 0.067868 accuracy= 0.937500
epoch : 10 step : 4289 loss = 0.029121 accuracy= 1.000000
epoch : 10 ste

epoch : 10 step : 4415 loss = 0.024703 accuracy= 1.000000
epoch : 10 step : 4416 loss = 0.049006 accuracy= 0.968750
epoch : 10 step : 4417 loss = 0.008587 accuracy= 1.000000
epoch : 10 step : 4418 loss = 0.028506 accuracy= 1.000000
epoch : 10 step : 4419 loss = 0.049514 accuracy= 0.968750
epoch : 10 step : 4420 loss = 0.059617 accuracy= 0.968750
epoch : 10 step : 4421 loss = 0.009607 accuracy= 1.000000
epoch : 10 step : 4422 loss = 0.026628 accuracy= 1.000000
epoch : 10 step : 4423 loss = 0.025619 accuracy= 1.000000
epoch : 10 step : 4424 loss = 0.141199 accuracy= 0.968750
epoch : 10 step : 4425 loss = 0.001576 accuracy= 1.000000
epoch : 10 step : 4426 loss = 0.008699 accuracy= 1.000000
epoch : 10 step : 4427 loss = 0.040567 accuracy= 0.968750
epoch : 10 step : 4428 loss = 0.175657 accuracy= 0.968750
epoch : 10 step : 4429 loss = 0.032752 accuracy= 1.000000
epoch : 10 step : 4430 loss = 0.147706 accuracy= 0.937500
epoch : 10 step : 4431 loss = 0.020740 accuracy= 1.000000
epoch : 10 ste

epoch : 10 step : 4557 loss = 0.023689 accuracy= 1.000000
epoch : 10 step : 4558 loss = 0.010426 accuracy= 1.000000
epoch : 10 step : 4559 loss = 0.028969 accuracy= 1.000000
epoch : 10 step : 4560 loss = 0.005364 accuracy= 1.000000
epoch : 10 step : 4561 loss = 0.025096 accuracy= 1.000000
epoch : 10 step : 4562 loss = 0.012049 accuracy= 1.000000
epoch : 10 step : 4563 loss = 0.014516 accuracy= 1.000000
epoch : 10 step : 4564 loss = 0.002124 accuracy= 1.000000
epoch : 10 step : 4565 loss = 0.024925 accuracy= 1.000000
epoch : 10 step : 4566 loss = 0.026398 accuracy= 1.000000
epoch : 10 step : 4567 loss = 0.027343 accuracy= 1.000000
epoch : 10 step : 4568 loss = 0.042562 accuracy= 1.000000
epoch : 10 step : 4569 loss = 0.001447 accuracy= 1.000000
epoch : 10 step : 4570 loss = 0.011321 accuracy= 1.000000
epoch : 10 step : 4571 loss = 0.044453 accuracy= 0.968750
epoch : 10 step : 4572 loss = 0.030597 accuracy= 1.000000
epoch : 10 step : 4573 loss = 0.014007 accuracy= 1.000000
epoch : 10 ste